In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
dataset = pd.read_csv('/content/drive/My Drive/_CENG442/SA/Test/TurkceSA/hepsiburada.csv')

In [5]:
dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


In [6]:
rating = dataset['Rating'].values.tolist()
review = dataset['Review'].values.tolist()

In [7]:
portion = int(len(rating)*0.8)

In [8]:
review_train = review[:portion]
review_test = review[portion:]
rating_train = rating[:portion]
rating_test = rating[portion:]

In [9]:
review_train[500]

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [10]:
review_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [11]:
# Tokenize the text data for training
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(review)

In [12]:
# Tokenize the text data for training
tokenizer.word_index

{'çok': 1,
 'bir': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'güzel': 7,
 'için': 8,
 'tavsiye': 9,
 'ederim': 10,
 'daha': 11,
 'ama': 12,
 'da': 13,
 'gayet': 14,
 'hızlı': 15,
 'teşekkürler': 16,
 'aldım': 17,
 'de': 18,
 'ürünü': 19,
 'gibi': 20,
 'yok': 21,
 'uygun': 22,
 'olarak': 23,
 'kaliteli': 24,
 'en': 25,
 '2': 26,
 'kargo': 27,
 'fiyat': 28,
 'elime': 29,
 'kadar': 30,
 'ile': 31,
 'göre': 32,
 'geldi': 33,
 'var': 34,
 'hepsiburada': 35,
 'ben': 36,
 'gerçekten': 37,
 '1': 38,
 'fiyata': 39,
 'gün': 40,
 'sonra': 41,
 'cok': 42,
 'kesinlikle': 43,
 'telefon': 44,
 'biraz': 45,
 'hiç': 46,
 'ulaştı': 47,
 'memnun': 48,
 'hem': 49,
 'değil': 50,
 'kullanışlı': 51,
 '3': 52,
 'mükemmel': 53,
 'oldu': 54,
 'kullanıyorum': 55,
 'önce': 56,
 'sipariş': 57,
 'tek': 58,
 'her': 59,
 'bence': 60,
 'harika': 61,
 'kalitesi': 62,
 'bi': 63,
 'ayrıca': 64,
 '5': 65,
 'teşekkür': 66,
 'fiyatı': 67,
 'olması': 68,
 'ne': 69,
 'herkese': 70,
 'bile': 71,
 'uzun': 72,
 'süper': 73,

In [13]:
# Tokenize the text data for training
review_train_tokens = tokenizer.texts_to_sequences(review_train)
review_test_tokens = tokenizer.texts_to_sequences(review_test)

In [14]:
review_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [15]:
print(review_train_tokens[800])

[19, 341, 52, 194, 30, 54, 7992, 55, 209, 603, 7887, 3, 36, 114, 164, 479, 85, 1, 1682, 782, 30, 2, 380, 326, 3, 1, 81, 132, 562, 9, 10]


In [16]:
num_tokens = [len(tokens) for tokens in review_train_tokens + review_test_tokens]
num_tokens = np.array(num_tokens)

In [17]:
np.mean(num_tokens)

20.744703220162876

In [18]:
np.max(num_tokens)

295

In [19]:
np.argmax(num_tokens)

21941

In [20]:
review_train[21941]

'Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...Bu cihazla

In [21]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens


59

In [22]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9597982726686571

In [23]:
review_train_pad = pad_sequences(review_train_tokens, maxlen=max_tokens)
review_test_pad = pad_sequences(review_test_tokens, maxlen=max_tokens)

In [24]:
np.array(review_train_tokens[800])

array([  19,  341,   52,  194,   30,   54, 7992,   55,  209,  603, 7887,
          3,   36,  114,  164,  479,   85,    1, 1682,  782,   30,    2,
        380,  326,    3,    1,   81,  132,  562,    9,   10])

In [25]:
review_train_pad[800]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,   19,  341,   52,  194,   30,
         54, 7992,   55,  209,  603, 7887,    3,   36,  114,  164,  479,
         85,    1, 1682,  782,   30,    2,  380,  326,    3,    1,   81,
        132,  562,    9,   10], dtype=int32)

In [26]:
review_train_pad.shape

(194797, 59)

In [27]:
review_test_pad.shape

(48700, 59)

In [28]:
# Tokenize the text data for training
index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in index.items()])

In [29]:
# Define a function to convert tokenized text back to human-readable text
def decode_review(tokens):
    return ' '.join([reverse_word_index.get(i, '?') for i in tokens])

In [30]:
review_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [31]:
# Define a function to convert tokenized text back to human-readable text
decode_review(review_train_tokens[800])

'ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum sürekli açık durur ve ben günde yaklaşık 12 saat çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim'

In [32]:
# Define the Sequential model with an Embedding layer
# The Embedding layer converts integer-encoded words into dense vectors of fixed size
model = Sequential()
embedding_size = 50
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [33]:
# Add GRU layers for sequence learning
# These layers help capture dependencies in the text data
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

In [34]:
# Compile the model with binary cross-entropy loss and Adam optimizer
# Binary cross-entropy is suitable for binary classification
optimizer = Adam(learning_rate = 1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_layer (Embedding)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
rating_train = np.array(rating_train)

In [39]:
# Train the model using the training dataset
# We use a batch size of 256 and train for 5 epochs
model.fit(review_train_pad, rating_train, epochs=5, batch_size=256)

Epoch 1/5
761/761 ━━━━━━━━━━━━━━━━━━━━ 16s 14ms/step - accuracy: 0.9443 - loss: 0.2374
Epoch 2/5
761/761 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.9733 - loss: 0.0843
Epoch 3/5
761/761 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9801 - loss: 0.0648
Epoch 4/5
761/761 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9855 - loss: 0.0498
Epoch 5/5
761/761 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9902 - loss: 0.0361


In [40]:
rating_test = np.array(rating_test)

In [41]:
# Evaluate the model on the test dataset to measure accuracy and loss
evaluation_result = model.evaluate(review_test_pad, rating_test)

1522/1522 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9370 - loss: 0.2085


In [42]:
print("Test accuracy:", evaluation_result[1])

Test accuracy: 0.9478439688682556


In [43]:
# Use the model to predict the sentiment of new samples
sample_predictions = model.predict(x=review_test_pad[0:1000]).T[0]

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [44]:
predicted_classes = np.array([1.0 if p > 0.5 else 0.0 for p in sample_predictions])
true_classes = np.array(rating_test[0:1000])

In [45]:
incorrect_predictions = np.where(predicted_classes != true_classes)[0]
print("Number of incorrect predictions:", len(incorrect_predictions))

Number of incorrect predictions: 59


In [48]:
sample_index = incorrect_predictions[0]
print("Index of incorrect prediction:", sample_index)
print("Text:", review_test[sample_index])
print("Predicted Rating:", sample_predictions[sample_index])
print("True Rating:", true_classes[sample_index])

Index of incorrect prediction: 5
Text: Ürün güzel ama teslimat çok geç yapıldı.
Predicted Rating: 0.97652286
True Rating: 0


In [49]:
new_texts = ["Gerçekten harika bir ürün ve elime hızlı ulaştı",
             "dün sipariş verdim ve bugün hemen elime ulaştı, kargo çok hızlı",
             "böyle büyük bir marka için tam bir hayal kırıklığı oldu",
             "harika",
             "sanki kullanılmış ürün göndermişler, bu satıcıyı kimseye tavsiye etmiyorum",
            "resimlerde gördüğünüzle elinize ulaşan arasında dağlar kadar fark var",
             "yorumlara bakıp az daha almayacaktım ama çok memnun kaldım",
            "Satıcı çok ilgisiz iade edeceğim kesinlikle",
             "tam bir fiyat performans ürünü",
             "hiç beklediğimi bulamadım"]

In [50]:
# Tokenize the text data for training
new_tokens = tokenizer.texts_to_sequences(new_texts)
new_tokens_pad = pad_sequences(new_tokens, maxlen=max_tokens)
print("Predictions for new texts:\n", model.predict(new_tokens_pad))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predictions for new texts:
 [[0.99816257]
 [0.9972427 ]
 [0.04948124]
 [0.99795437]
 [0.00404574]
 [0.9951945 ]
 [0.98699784]
 [0.0246093 ]
 [0.9980774 ]
 [0.39965066]]
